In [71]:
# Build individual onset file for spm analysis and save it in fmri preprocess folder
# Import
import os
import pandas as pd

# Set path
path_root = "/gs/project/gsf-624-aa/HCP/HCP_raw_data"
path_out = "/gs/project/gsf-624-aa/HCP/fmri_preprocess_all_tasks_niak-fix-scrub_900R"

# Loop over subject and runs and create individual onset
list_subj = os.listdir(path_root)
# loop over runs
for run in 'LR','RL':
    for index, subj in enumerate(list_subj):
        path_eprime = os.path.join(path_root,subj,"unprocessed/3T/tfMRI_WM_{}/LINKED_DATA/EPRIME/EVs".format(run))
        if not os.path.exists(path_eprime):
            print('Subject %s has no Eprime folder for WM_%s task' % (subj, run))
            continue
        
        # get trial files
        trial_files = os.listdir(path_eprime)
        dump_tmp = [trial_files.remove(ii) for ii in ('WM_Stats.csv','Sync.txt','0bk_nlr.txt','2bk_nlr.txt','0bk_err.txt','0bk_cor.txt','2bk_err.txt','2bk_cor.txt','all_bk_err.txt','all_bk_cor.txt')]
        
        # Create subject EVs folder
        eprime_out = os.path.join(path_out,'EVs','HCP{}'.format(subj),'WM_{}'.format(run))
        if not os.path.exists(eprime_out):
            os.makedirs(eprime_out)
        
        #Loop over trials and buid onset file
        eprime_final = pd.DataFrame()
        for trial_f in trial_files :
            trial = trial_f[:-4] 
            eprime = pd.read_csv(os.path.join(path_eprime,trial_f), sep='\t',header=None)
            
            # Add onset_trial name
            onset_list = [trial.format(row) for ind, row in eprime.iterrows()]
            eprime.loc[:,''] = onset_list
            
            # re-arrange colomns
            cols = eprime.columns.tolist()
            cols = cols[-1:] + cols[:-1]
            eprime = eprime[cols]
            
            # append to other trial
            eprime_final = eprime_final.append(eprime)
        
        # create new contrast    
        eprime_raw = eprime_final
        
        # Create '0back' trial
        sub_select = eprime_raw[eprime_raw[''].str.contains('0bk')]
        sub_select.iloc[:,0] = [x.replace(x,'0bk') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create '2back' trial
        sub_select = eprime_raw[eprime_raw[''].str.contains('2bk')]
        sub_select.iloc[:,0] = [x.replace(x,'2bk') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'faces' trial
        sub_select = eprime_raw[eprime_raw[''].str.contains('faces')]
        sub_select.iloc[:,0] = [x.replace(x,'faces') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'other_faces' trial
        sub_select = eprime_raw[~eprime_raw[''].str.contains('faces')]
        sub_select.iloc[:,0] = [x.replace(x,'other_faces') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'places' trial
        sub_select = eprime_raw[eprime_raw[''].str.contains('places')]
        sub_select.iloc[:,0] = [x.replace(x,'places') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'other_places' trial
        sub_select = eprime_raw[~eprime_raw[''].str.contains('places')]
        sub_select.iloc[:,0] = [x.replace(x,'other_places') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'tools' trial
        sub_select = eprime_raw[eprime_raw[''].str.contains('tools')]
        sub_select.iloc[:,0] = [x.replace(x,'tools') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'other_tools' trial
        sub_select = eprime_raw[~eprime_raw[''].str.contains('tools')]
        sub_select.iloc[:,0] = [x.replace(x,'other_tools') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'body' trial
        sub_select = eprime_raw[eprime_raw[''].str.contains('body')]
        sub_select.iloc[:,0] = [x.replace(x,'body') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Create 'other_body' trial
        sub_select = eprime_raw[~eprime_raw[''].str.contains('body')]
        sub_select.iloc[:,0] = [x.replace(x,'other_body') for x in sub_select[''].get_values()]
        eprime_final = eprime_final.append(sub_select)
        
        # Put header
        eprime_final.rename(columns={0:'start',1:'duration',2:'repetition'},inplace=True)
        eprime_final = eprime_final.reset_index(drop=True)
        
        # Save final onset file
        eprime_final.to_csv(os.path.join(eprime_out,"WM_{}_spm_onset.csv".format(run)),index=False)
